In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import keras_ocr
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import pandas as pd
from tensorflow.keras.applications import ResNet50 as PT_Model
# from tensorflow.keras.applications import VGG16 as PT_Model

In [ ]:
# rec = keras_ocr.recognition.Recognizer()
# rec.compile()

In [ ]:
image_size = (32, 200)
MAX_LABEL_LENGTH = 50
FREEZE_PT_MODEL = False

In [ ]:
alphabet = (' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', \
            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', \
            'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'Å', 'Ä', 'Ö',\
            '-', ',', '/'
           )
to_alphabet = {i:key for i, key in enumerate(alphabet)}
to_class = {key:i for i,key in enumerate(alphabet)}

N_CLASSES = len(alphabet)

## Load Data

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/data_2/poly_crops",
    validation_split=0.2,
    subset="training",
    label_mode=None,
    seed=None,
    shuffle=False,
    batch_size=None,
    image_size=image_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/data_2/poly_crops",
    validation_split=0.2,
    subset="validation",
    label_mode=None,
    seed=None,
    shuffle=False,
    batch_size=None,
    image_size=image_size
)

In [ ]:
def scale_0_1(data):
    return data / 255

def to_gray_scale(data):
    return data[:,:,0:1]

In [ ]:
#train_ds = train_ds.map(scale_0_1)
# #train_ds = train_ds.map(to_gray_scale)

#val_ds = val_ds.map(scale_0_1)
# #val_ds = val_ds.map(to_gray_scale)

### Labels

In [ ]:
df = pd.read_csv('dataset/data_2/poly_labels.csv', header=None)
train_df = df.values[:len(train_ds)]
val_df = df.values[len(train_ds):]

In [ ]:
train_df

In [ ]:
def format_labels(labels, max_len=50, n_classes=37):
    labels = labels.astype(str)
    max_length_found = np.max([len(s[0]) for s in labels])
    if (max_length_found > max_len):
        print(f'WARNING! Labels are truncated to {max_len} characters.')
    out = np.zeros((labels.shape[0], max_len, n_classes), dtype=int)

    for i,label in enumerate(labels):
        label = np.char.upper(label)[0]
        for j,char in enumerate(label):
            try:
                out[i, j, to_class[char]] = 1
            except KeyError:
                pass #out[i, j, to_class[' ']] =  1 # not a character in alphabet
        for k in range(j+1,max_len):
            out[i, k, 0] = 1
    return out

train_df = format_labels(train_df, max_len=MAX_LABEL_LENGTH, n_classes=N_CLASSES)
val_df = format_labels(val_df, max_len=MAX_LABEL_LENGTH, n_classes=N_CLASSES)

In [ ]:
train_lbl = tf.data.Dataset.from_tensor_slices(train_df)
val_lbl = tf.data.Dataset.from_tensor_slices(val_df)

In [ ]:
for a in train_lbl.take(1):
    print(a[0])

In [ ]:
BATCH_SIZE = 32

train_ds = tf.data.Dataset.zip((train_ds, train_lbl))
train_ds = train_ds.batch(BATCH_SIZE)

val_ds = tf.data.Dataset.zip((val_ds, val_lbl))
val_ds = val_ds.batch(BATCH_SIZE)

In [ ]:
# for a,b in train_ds:
#     print(a.shape)
#     print(b.shape)
#     break

In [ ]:
train_lbl

## Model

In [ ]:
pt_model = PT_Model(
    include_top=False,
    # weights='imagenet',
    input_shape=(*image_size, 3),
)
# import tensorflow as tf

# from tensorflow.keras import datasets, layers, models
# import matplotlib.pyplot as plt

pt_input = pt_model.layers[0].input
pt_output = pt_model.get_layer(name='conv3_block1_out').output
pt_model = tf.keras.Model(pt_input, pt_output)
# pt_model = models.Sequential()
# pt_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(*image_size, 3)))
# pt_model.add(layers.MaxPooling2D((2, 2)))
# pt_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# pt_model.add(layers.MaxPooling2D((2, 2)))
# pt_model.add(layers.Conv2D(64, (3, 3), activation='relu'))


def make_head(feature_shape, n_char = 50, n_classes = 37):
    inputs = tf.keras.Input(shape=feature_shape)
    x = layers.BatchNormalization()(inputs)
    x = layers.Reshape((feature_shape[-2]*feature_shape[-3], feature_shape[-1]))(x)
    x = layers.Conv1D(1024, 3, strides=2, padding='same')(x)
    x = layers.Conv1D(256,3,strides=1, padding='same')(x)
    # print(x.shape)
    x = layers.Flatten()(x)
    # print(x.shape)
    x = layers.Dense(n_char * n_classes)(x)
    outputs = layers.Reshape((n_char, n_classes))(x)
    outputs = layers.Softmax(axis=-2)(outputs)
    # # outputs = layers.LSTM(n_classes, activation='softmax', return_sequences=True)(x)
    # outputs = layers.LSTM(n_classes, activation='softmax', return_sequences=True)(x)
    # print(outputs.shape)
    return tf.keras.Model(inputs, outputs)

pt_model.trainable = not FREEZE_PT_MODEL
head = make_head(pt_model.output_shape[1:], n_char = MAX_LABEL_LENGTH, n_classes=N_CLASSES)
head.summary()

In [ ]:
N_CLASSES

In [ ]:
pt_model.output_shape[1:]

In [ ]:
pt_model.summary()

In [ ]:
model = tf.keras.Sequential([
    pt_model,
    head
])

model.summary()

## Train model

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()]
)

model.fit(train_ds, validation_data=val_ds, epochs=10)

In [ ]:
model_name = 'final_model1'
if FREEZE_PT_MODEL:
    model_name += '_frozen'

model.save(f'model/{model_name}.h5')

In [ ]:
test_ds = val_ds.unbatch()
for img, label in test_ds.take(-1):
#    print(img.shape)
    y = np.argmax(label, 1)
    plt.imshow(img / 256)
    plt.axis('off')
    plt.show()
    plt.close()
    
    y_ = model(np.reshape(img, (1, 32, 200, 3)))
    y_ = np.argmax(y_, 2)
#    print(y_.shape)
#    print(y.shape)
    print(''.join([to_alphabet[a] for a in y]))
    print(''.join([to_alphabet[a] for a in y_[0]]))